#### Challenge 43: DSA key recovery from nonce

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [4]:
from Crypto.Util import number
from Crypto.Random import random
from Crypto.Hash.SHA256 import SHA256Hash

import cryptopals as cp
import sha1

<div class="alert alert-block alert-info">

**Step 1**: Relocate so that you are out of easy travel distance of us.

**Step 2**: Implement DSA, up to signing and verifying, including parameter generation.

Hah-hah you're too far away to come punch us.

Just kidding you can skip the parameter generation part if you want; if you do, use these params:

```
p = 800000000000000089e1855218a0e7dac38136ffafa72eda7
    859f2171e25e65eac698c1702578b07dc2a1076da241c76c6
    2d374d8389ea5aeffd3226a0530cc565f3bf6b50929139ebe
    ac04f48c3c84afb796d61e5a4f9a8fda812ab59494232c7d2
    b4deb50aa18ee9e132bfa85ac4374d7f9091abc3d015efc87
    1a584471bb1
```

```
q = f4f47f05794b256174bba6e9b396a7707e563c5b
```
    
``` 
g = 5958c9d3898b224b12672c0b98e06c60df923cb8bc999d119
    458fef538b8fa4046c8db53039db620c094c9fa077ef389b5
    322a559946a71903f990f1f7e0e025e2d7f7cf494aff1a047
    0f5b64c36b625a097f1651fe775323556fe00b3608c887892
    878480e99041be601a62166ca6894bdd41a7054ec89f756ba
    9fc95302291
```
</div>

In [5]:
p = 0x800000000000000089e1855218a0e7dac38136ffafa72eda7859f2171e25e65eac698c1702578b07dc2a1076da241c76c62d374d8389ea5aeffd3226a0530cc565f3bf6b50929139ebeac04f48c3c84afb796d61e5a4f9a8fda812ab59494232c7d2b4deb50aa18ee9e132bfa85ac4374d7f9091abc3d015efc871a584471bb1
q = 0xf4f47f05794b256174bba6e9b396a7707e563c5b
g = 0x5958c9d3898b224b12672c0b98e06c60df923cb8bc999d119458fef538b8fa4046c8db53039db620c094c9fa077ef389b5322a559946a71903f990f1f7e0e025e2d7f7cf494aff1a0470f5b64c36b625a097f1651fe775323556fe00b3608c887892878480e99041be601a62166ca6894bdd41a7054ec89f756ba9fc95302291


From **Applied Cryptography** (Schneier), to sign a message `m`:

Private Key:  `x` = an number < `q`  
Public Key:   `y = g^x mod p`

1. Alice generates a random number, `k`, less than `q`
2. Alice generates:   
    `r = (g^k mod p) mod q`   
    `s = (k^-1 (H(m) + xr)) mod q`
3. Send parameters `r` and `s` to Bob as Alice's signature
4. Bob verifies the signature by computing:
    - `w = s^-1 mod q`
    - `u1 = (H(m) * w) mod q`
    - `u2 = (rw) mod q`
    - `v = ((g^u1 * y^u2) mod p) mod q`
5. If `v==r` the signature is verified

In [6]:
x = random.randint(0, q-1)
y = pow(g, x, p)

def gen_DSA_sig(x, m, p, q, g):

    k = random.randint(0, q-1)
    
    r = pow(g, k, p) % q
    sha_out = sha1.SHA1(m).finish()
    sha_int = int(sha_out.hex(), 16)
    s = (cp.invmod(k, q) * (sha_int + x*r)) % q
    
    return(r, s, k)

def check_DSA_sig(m, y, r, s, p, g, q):
    
    w = cp.invmod(s, q)
    sha_out = sha1.SHA1(m).finish()
    sha_int = int(sha_out.hex(), 16)
    u1 = (sha_int * w) % q
    u2 = (r*w) % q
    v = ((pow(g, u1, p) * pow(y, u2, p)) % p) % q
    
    print(hex(v))
    print(hex(r))
    if v==r:
        return(True)
    else:
        return(False)

In [7]:
message = b'Test Msg'
[r, s, _] = gen_DSA_sig(x, message, p, q, g)
check_DSA_sig(message, y, r, s, p, g, q)

0xd914c4a610dd1782234694d8bbb60364e24b079a
0xd914c4a610dd1782234694d8bbb60364e24b079a


True

<div class="alert alert-block alert-info">
    
("But I want smaller params!" Then generate them yourself.)

The DSA signing operation generates a random subkey `k`. You know this because you implemented the DSA sign operation.

This is the first and easier of two challenges regarding the DSA `k` subkey.

Given a known `k`, it's trivial to recover the DSA private key `x`:

          (s * k) - H(msg)
      x = ----------------  mod q
                  r

Do this a couple times to prove to yourself that you grok it. Capture it in a function of some sort.

</div>

In [8]:
def dsa_priv_key_from_k(m, k, r, s):

    sha_out = sha1.SHA1(m).finish()
    H_m = int(sha_out.hex(), 16)
    
    x_guess = ((((s*k) - H_m)) * cp.invmod(r, q)) % q
    
    return(x_guess)

In [9]:
message = b'Recover My Key'
[r, s, k] = gen_DSA_sig(x, message, p, q, g)

x_guess = dsa_priv_key_from_k(message, k, r, s)

if x_guess == x:
    print('Private key SUCCESSFULLY recovered')
else:
    print('Key recovery FAILED')

Private key SUCCESSFULLY recovered


<div class="alert alert-block alert-info">
    
Now then. I used the parameters above. I generated a keypair. My pubkey is:

```
  y = 84ad4719d044495496a3201c8ff484feb45b962e7302e56a392aee4
      abab3e4bdebf2955b4736012f21a08084056b19bcd7fee56048e004
      e44984e2f411788efdc837a0d2e5abb7b555039fd243ac01f0fb2ed
      1dec568280ce678e931868d23eb095fde9d3779191b8c0299d6e07b
      bb283e6633451e535c45513b2d33c99ea17
```
    
I signed

```
For those that envy a MC it can be hazardous to your health  
So be friendly, a matter of life and death, just like a etch-a-sketch
```

<br>

My SHA1 for this string was `d2d0714f014a9784047eaeccf956520045c45265` 
    
I don't know what NIST wants you to do, but when I convert that hash to an integer I get: `0xd2d0714f014a9784047eaeccf956520045c45265`.

I get:

  `r = 548099063082341131477253921760299949438196259240`  
  `s = 857042759984254168557880549501802188789837994940`

I signed this string with a broken implemention of DSA that generated `"k"` values between `0` and `2^16`. What's my private key?

Its SHA-1 fingerprint (after being converted to hex) is:

`0954edd5e0afe5542a4adf012611a91912a3ec16`

Obviously, it also generates the same signature for that string.

</div>

In [10]:
pub_key = int('84ad4719d044495496a3201c8ff484feb45b962e7302e56a392aee4' + \
              'abab3e4bdebf2955b4736012f21a08084056b19bcd7fee56048e004' + \
              'e44984e2f411788efdc837a0d2e5abb7b555039fd243ac01f0fb2ed' + \
              '1dec568280ce678e931868d23eb095fde9d3779191b8c0299d6e07b' + \
              'bb283e6633451e535c45513b2d33c99ea17', 16)

message = b'For those that envy a MC it can be hazardous to your health\n' + \
          b'So be friendly, a matter of life and death, just like a etch-a-sketch\n'

cryptopals_sha = 0xd2d0714f014a9784047eaeccf956520045c45265
my_sha = int((sha1.SHA1(message).finish()).hex(), 16)

if (my_sha == cryptopals_sha):
    print('Our hashes match..continue')
else:
    raise Exception
    

Our hashes match..continue


In [11]:
def gen_DSA_sig_given_k(x, m, p, q, g, k):
   
    r = pow(g, k, p) % q
    sha_out = sha1.SHA1(m).finish()
    sha_int = int(sha_out.hex(), 16)
    s = (cp.invmod(k, q) * (sha_int + x*r)) % q
    
    return(r, s)

In [12]:
r = 548099063082341131477253921760299949438196259240
s = 857042759984254168557880549501802188789837994940
true_fp = '0954edd5e0afe5542a4adf012611a91912a3ec16'

key_found = False
for k in range(2**16):
    
    x_guess = dsa_priv_key_from_k(message, k, r, s)
    [r_guess, s_guess] = gen_DSA_sig_given_k(x_guess, message, p, q, g, k)
    
    if (r_guess==r) and (s_guess == s):
        key_found = True
        break

if not(key_found):
    print('Key not found...try again')
else:
    print('Found private key:')
    print(f'x = {x_guess}')
    print(f'x.hex = {hex(x_guess)}')
    print(f'k = {k}')

Found private key:
x = 125489817134406768603130881762531825565433175625
x.hex = 0x15fb2873d16b3e129ff76d0918fd7ada54659e49
k = 16575


In [13]:
x_guess_bytes = hex(x_guess)[2:].encode()
guessed_fp = sha1.SHA1(x_guess_bytes).finish().hex()
if guessed_fp == true_fp:
    print('Fingerprint matches')
else:
    print('Private key fingerprint does NOT match')

Fingerprint matches


[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)